# В этом файле когда-то будет парсер

Необходимо спарсить: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow

In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

In [41]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Атдыхаем, ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_15656\60259596.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_15656\60259596.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

In [15]:
df = pd.DataFrame()

In [27]:
def table_parser(soup: BeautifulSoup):
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link}

        global df
        df = df.append(data, ignore_index=True)

In [28]:
for _ in tqdm(range(100)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_parser(soup)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

 55%|█████▌    | 55/100 [10:21<08:28, 11.30s/it]


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00988893+48451]
	(No symbol) [0x0091B8A1]
	(No symbol) [0x00825058]
	(No symbol) [0x00818205]
	(No symbol) [0x0081810C]
	(No symbol) [0x00816D46]
	(No symbol) [0x008175CA]
	(No symbol) [0x0082E8A9]
	(No symbol) [0x008291EA]
	(No symbol) [0x00828FC7]
	(No symbol) [0x0087DD16]
	(No symbol) [0x0086A2BC]
	(No symbol) [0x0087C482]
	(No symbol) [0x0086A0B6]
	(No symbol) [0x00847E08]
	(No symbol) [0x00848F2D]
	GetHandleVerifier [0x00BE8E3A+2540266]
	GetHandleVerifier [0x00C28959+2801161]
	GetHandleVerifier [0x00C2295C+2776588]
	GetHandleVerifier [0x00A12280+612144]
	(No symbol) [0x00924F6C]
	(No symbol) [0x009211D8]
	(No symbol) [0x009212BB]
	(No symbol) [0x00914857]
	BaseThreadInitThunk [0x76F47D59+25]
	RtlInitializeExceptionChain [0x77A1B74B+107]

In [35]:
(~df.Link.isna()).sum()

1755

In [38]:
Links = df.Link.tolist()

In [50]:
characteristics_df = pd.DataFrame()

In [ ]:
for cur_link in tqdm(Links):
        if cur_link!= None:
            driver.get(cur_link)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(randint(1, 5))
            html = driver.page_source
            soup2 = BeautifulSoup(html, 'html.parser')
            title_tag = soup2.find('h1', {'class': 'x-item-title__mainTitle'})
            title_text = title_tag.text.strip()
            parent_element = driver.find_element(By.CLASS_NAME, 'ux-layout-section--features')
            characteristics = parent_element.text.split("\n")
            characteristics_data = {"Title": title_text}
            for i in range(len(characteristics)):
                if i%2==0:
                    name = characteristics[i]
                else:
                    value = characteristics[i]
                    characteristics_data[name] = value

            characteristics_df = characteristics_df.append(characteristics_data, ignore_index=True)
        pass

  0%|          | 1/2800 [00:07<6:09:41,  7.92s/it]